# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [ ]:
# Load pickled data
import pickle

#loading data set, not available in the github repository due to size constraints, Check README and/or Writeup
training_file = 'traffic-signs-data/train.p'
validation_file= 'traffic-signs-data/valid.p'
testing_file = 'traffic-signs-data/test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']


---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [ ]:
import numpy as np 

#Number of training examples
n_train = len(X_train)

#Number of validation examples
n_validation = len(X_valid)

#Number of testing examples.
n_test = len(X_test)

#What's the shape of an traffic sign image?
image_shape = X_test[0].shape

#number of unique classes/labels there are in the dataset.
n_classes = np.amax([np.amax(y_train), np.amax(y_valid), np.amax(y_test)]) - np.amin([np.amin(y_train), np.amin(y_valid), np.amin(y_test)]) + 1


print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc. 

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections. It can be interesting to look at the distribution of classes in the training, validation and test set. Is the distribution the same? Are there more examples of some classes than others?

In [ ]:
### Data exploration visualization
import matplotlib.pyplot as plt
import random
import math 

n_visualize = 9 #number of images to be plotted
n_columns = 3 #number of columns in the figure
n_rows = math.ceil(n_visualize/n_columns) #number of rows in the figure

plt.figure(figsize=(15,15))
plt.suptitle('Training set samples', fontsize=16)

for i in range(n_visualize):
    index = random.randint(0,n_train-1)
    plt.subplot(n_rows, n_columns, i+1)
    plt.imshow(X_train[index])
    plt.title(('image:'+ str(index) +', label = ' + str(y_train[index])))

    
# Visualizations will be shown in the notebook.
#%matplotlib inline
plt.subplots_adjust(top=0.95)
plt.savefig('Images/sample_data.jpg')
plt.show()

In [ ]:
#show number of examples for each label in training set
labelCounter = [0] * n_classes

for i in range(n_train):
    labelCounter[y_train[i]] += 1

plt.figure(figsize=(15,15))
plt.bar(range(n_classes), labelCounter)
plt.title('Number of occurance of each label in training set')
plt.xlabel('label')
plt.ylabel('repitations')

plt.savefig('Images/training_set_bar.jpg')
plt.show()

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play! 

With the LeNet-5 solution from the lecture, you should expect a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission. 

There are various aspects to consider when thinking about this problem:

- Neural network architecture (is the network over or underfitting?)
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

In [ ]:
### Priod to preprocessing the data, augmentation is applied

#defining functions to perfrom data augmentation on X_train
import cv2
import random
import math


def augmentimg(image2aug):
    #random msg to select which selector to use
    method_selector = random.randint(1,3)
    
    if(method_selector == 1):
        #Apply prespective transfromation
        
        #getting random vertices of the image
        x1 = random.randint(0, math.ceil(image_shape[0]/6))
        x2 = random.randint(math.ceil(5 * image_shape[0]/6), image_shape[0])
        y1 = random.randint(0, math.ceil(image_shape[1]/6))
        y2 = random.randint(math.ceil(5 * image_shape[1]/6), image_shape[0])
        
        #applying prespective transformation
        pts1 = np.float32([[x1,y1],[x2,y1],[x1,y2],[x2,y2]])
        pts2 = np.float32([[0,0],[image_shape[0],0],[0,image_shape[1]],[image_shape[0],image_shape[1]]])
        M = cv2.getPerspectiveTransform(pts1,pts2)
        output_img = cv2.warpPerspective(image2aug,M,(image_shape[0],image_shape[1]))
        
    elif(method_selector == 2):
        #apply translation
        
        #getting random shifts for the image
        shift_x = random.randint(-math.ceil(image_shape[0]/6), math.ceil(image_shape[0]/6))
        shift_y = random.randint(-math.ceil(image_shape[1]/6), math.ceil(image_shape[1]/6))
        
        #applying translation
        M = np.float32([[1,0,shift_x],[0,1,shift_y]])
        output_img = cv2.warpAffine(image2aug,M,(image_shape[0],image_shape[1]))
        
    elif(method_selector == 3):
        #apply rotation
        
        #obtaining random angle to rotate
        angle2rotate = random.randint(-10, 10)
        
        #applying rotation
        M = cv2.getRotationMatrix2D((math.ceil(image_shape[0]/2),math.ceil(image_shape[1]/2)),angle2rotate,1)
        output_img = cv2.warpAffine(image2aug,M,(image_shape[0],image_shape[1]))
        
    return output_img

In [ ]:
#Applying data augmentation 

#minimum number of samples for each label
aug_min = 250

#number of augmented editions to each label if they exceed the minimum
n_aug = 20

#list for the indices of original image and the augmented version
###Note this will become meaningless one data is shuffled
org2aug_index = []


for i in range(n_classes):
    #creating augmented versions
    while(labelCounter[i] < aug_min):
        indices = []
        indices = np.where(y_train==i)[0]
        index2aug = random.choice(indices)
        img2aug = X_train[index2aug]
        X_train = np.insert(X_train, n_train, augmentimg(img2aug), axis=0)
        y_train = np.insert(y_train, n_train, i, axis=0)
        org2aug_index.append([index2aug, n_train])
        labelCounter[i] += 1
        n_train += 1  
    else:
        for j in range(n_aug):
            indices = []
            indices = np.where(y_train==i)[0]
            index2aug = random.choice(indices)
            img2aug = X_train[index2aug]
            X_train = np.insert(X_train, n_train, augmentimg(img2aug), axis=0)
            y_train = np.insert(y_train, n_train, i, axis=0)
            org2aug_index.append([index2aug, n_train])
            labelCounter[i] += 1
            n_train += 1   


In [ ]:
x = np.array([1,2,3,4,5,6,5,4,3,2,1])
print(x)
z = (x==2).nonzero()[0]
print(z)
y = random.choice(z)
print(y)

In [ ]:
#Show random samples of original training set images and augmented versions

plt.figure(figsize=(15,15))
plt.suptitle('Data Augmentation Sample', fontsize=16)

#number of original/augmented samples to show
n_show = 4

for i in range(n_show):
    #obtain random original and augmented image
    index = random.randint(0,len(org2aug_index)-1)
    org2aug = org2aug_index[index]
    org_img = X_train[org2aug[0]]
    aug_img = X_train[org2aug[1]]
    
    #show the images
    plt.subplot(n_show,2, (2*i)+1)
    plt.imshow(org_img)
    plt.title('Original Image {} {}'.format(i+1, y_train[org2aug[0]]))
    plt.subplot(n_show,2, (2*i)+2)
    plt.imshow(aug_img)
    plt.title('Augmented Image {}. {}'.format(i+1, y_train[org2aug[1]]-1))
    
plt.subplots_adjust(top=0.95)
plt.savefig('Images/data_aug_sample.jpg')
plt.show()    

In [ ]:
#show number of examples for each label in training set after augmentation
labelCounter = [0] * n_classes

for i in range(n_train):
    labelCounter[y_train[i]] += 1

plt.figure(figsize=(15,15))
plt.bar(range(n_classes), labelCounter)
plt.title('Number of occurance of each label in training set after augmentation')
plt.xlabel('label')
plt.ylabel('repitations')

plt.savefig('Images/training_augmented_set_bar.jpg')
plt.show()

### Pre-process the Data Set (normalization, grayscale, etc.)

Minimally, the image data should be normalized so that the data has mean zero and equal variance. For image data, `(pixel - 128)/ 128` is a quick way to approximately normalize the data and can be used in this project. 

Other pre-processing steps are optional. You can try different techniques to see if it improves performance. 

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

In [ ]:
#Defining a function which accepts a list of rgb images and returns a list of grayscaled images
#this function returns a Heigth x Width x 1 image
import cv2

def rgb2gray(rgb_list):
    imgGray = []
    for img in rgb_list:
        grayCh = np.zeros([img.shape[0], img.shape[1], 1])
        grayCh[:,:,0] = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        imgGray.append(grayCh)
    return imgGray


In [ ]:
#Show Grayscaling samples

plt.figure(figsize=(15,15))
plt.suptitle('Grayscaling sample', fontsize=16)

#number of original/grayscales samples to show
n_show = 4

for i in range(n_show):
    #obtain random image
    index = random.randint(0,n_train-1)
    img = X_train[index]
    
    #show the original/grayscaled images
    plt.subplot(n_show,2, (2*i)+1)
    plt.imshow(img)
    plt.title('Original Image {}'.format(i+1))
    plt.subplot(n_show,2, (2*i)+2)
    #print(rgb2gray([img])[0][:,:,0].shape)
    plt.imshow(rgb2gray([img])[0][:,:,0], cmap='gray')
    plt.title('Grayscaled Image {}'.format(i+1))

plt.subplots_adjust(top=0.95)
plt.savefig('Images/data_gray_sample.jpg')
plt.show()   

In [ ]:
#defining a function to normalize data
def normalize(img_list):
    mean = 128
    std = 128
    norm_list = []
    for img in img_list:
        norm_list.append(np.divide(img - mean, std))
    return norm_list

In [ ]:
### Preprocess the data here. 

#shuffling the data
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)
X_train, y_train = shuffle(X_train, y_train)

convToGray = True #bolean to determine whether to convert to grayscale

#normalizing the image 
if(convToGray & (image_shape[2] == 3)): 
    #apply_grayscaling prior to normalizing
    X_train_n = normalize(rgb2gray(X_train))
    X_valid_n = normalize(rgb2gray(X_valid))
    X_test_n = normalize(rgb2gray(X_test))
else:
    #only normalizing the images
    X_train_n = normalize(X_train)
    X_valid_n = normalize(X_valid)
    X_test_n = normalize(X_test)   
    
image_shape = X_train_n[0].shape

### Model Architecture

In [ ]:
#tensorflow setup
import tensorflow as tf

EPOCHS = 20
BATCH_SIZE = 256

In [ ]:
### Defining the first architecture here.
#####LeNet architecture
###### Note: The LeNet architecture is slightly modified here 
###### by adding dropout functionality to the fully connvected layers

from tensorflow.contrib.layers import flatten

def LeNet(x, keep_prob):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    
    inheight, inwidth, inchannels = image_shape
        
    #Layer 1: Convolutional. Input = image. Output = 28x28x6.
    wc1height = inheight - 28 + 1
    wc1width = inwidth - 28 + 1
    wc1 = tf.Variable(tf.truncated_normal([wc1height, wc1width, inchannels, 6], dtype=tf.float32, mean=mu, stddev=sigma),\
                      name='LeNet-wc1')
    bc1 = tf.Variable(tf.zeros([6], dtype=tf.float32), name='LeNet-bc1')
    strides = [1, 1, 1, 1]
    padding = 'VALID'
    out_layer1 = tf.nn.bias_add(tf.nn.conv2d(x, wc1, strides, padding), bc1, name='LeNet-out_layer1')
    
    #Activation.
    out_layer1 = tf.nn.relu(out_layer1, name='LeNet-out_layer1-relu')
    
    #Pooling. Input = 28x28x6. Output = 14x14x6.
    out_layer1 = tf.nn.max_pool(out_layer1, [1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name='LeNet-out_layer1-pool')
    
    #Layer 2: Convolutional. Output = 10x10x16.
    wc2 = tf.Variable(tf.truncated_normal([5, 5, 6, 16], dtype=tf.float32, mean=mu, stddev=sigma), name='LeNet-wc2')
    bc2 = tf.Variable(tf.zeros([16], dtype=tf.float32), name='LeNet-bc2')
    strides = [1, 1, 1, 1]
    padding = 'VALID'
    out_layer2 = tf.nn.bias_add(tf.nn.conv2d(out_layer1, wc2, strides, padding), bc2, name='LeNet-out_layer2')
    
    #Activation.
    out_layer2 = tf.nn.relu(out_layer2, name='LeNet-out_layer2-relu')
    
    #Pooling. Input = 10x10x16. Output = 5x5x16.
    out_layer2 = tf.nn.max_pool(out_layer2, [1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name='LeNet-out_layer2-pool')

    #Flatten. Input = 5x5x16. Output = 400.
    out_conv_layers = flatten(out_layer2)
    
    #Layer 3: Fully Connected. Input = 400. Output = 120.
    wf1 = tf.Variable(tf.truncated_normal([400, 120], dtype=tf.float32, mean=mu, stddev=sigma), name='LeNet-wf1')
    bf1 = tf.Variable(tf.zeros([120], dtype=tf.float32), name='LeNet-bf1')
    out_layer3 = tf.add(tf.matmul(out_conv_layers, wf1), bf1, name='LeNet-out_layer3')
    
    #Activation.
    out_layer3 = tf.nn.relu(out_layer3, name='LeNet-out_layer3-relu')
    
    #Dropout
    out_layer3 = tf.nn.dropout(out_layer3, keep_prob, name='LeNet-out_layer3-dropout')
    
    
    #Layer 4: Fully Connected. Input = 120. Output = 84.
    wf2 = tf.Variable(tf.truncated_normal([120, 84], dtype=tf.float32, mean=mu, stddev=sigma), name='LeNet-wf2')
    bf2 = tf.Variable(tf.zeros([84], dtype=tf.float32), name='LeNet-bf2')
    out_layer4 = tf.add(tf.matmul(out_layer3, wf2), bf2, name='LeNet-out_layer4')
    
    #Activation.
    out_layer4 = tf.nn.relu(out_layer4, name='LeNet-out_layer4-relu')
    
    #Droput
    out_layer4 = tf.nn.dropout(out_layer4, keep_prob, name='LeNet-out_layer4-dropout')
    
    #Layer 5: Fully Connected. Input = 84. Output = n_classes.
    wf3 = tf.Variable(tf.truncated_normal([84, n_classes], dtype=tf.float32, mean=mu, stddev=sigma), name='LeNet-wf3')
    bf3 = tf.Variable(tf.zeros([n_classes], dtype=tf.float32), name='LeNet-bf3')
    logits = tf.add(tf.matmul(out_layer4, wf3), bf3)
    
    return logits

In [ ]:
### Defining the second architecture here.
#####This architecture is sumilar consists of two convolutional layers 
#####whose output is feed to a two stage fully connected layer
#####in a smilar approach to the one described in 
##### "http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf"

from tensorflow.contrib.layers import flatten

def s2_Covnet(x, keep_prob):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    
    inheight, inwidth, inchannels = image_shape
        
    #Layer 1: Convolutional. Input = image. Output = 28x28x30.
    wc1height = inheight - 28 + 1
    wc1width = inwidth - 28 + 1
    wc1 = tf.Variable(tf.truncated_normal([wc1height, wc1width, inchannels, 30], dtype=tf.float32, mean=mu, stddev=sigma),\
                     name='s2_Covnet-wc1')
    bc1 = tf.Variable(tf.zeros([30], dtype=tf.float32), name='s2_Covnet-bc1')
    strides = [1, 1, 1, 1]
    padding = 'VALID'
    out_layer1 = tf.nn.bias_add(tf.nn.conv2d(x, wc1, strides, padding), bc1, name='s2_Covnet-out_layer1')
    
    #Activation.
    out_layer1 = tf.nn.relu(out_layer1, name='s2_Covnet-out_layer1-relu')
    
    #Pooling. Input = 28x28x30 Output = 14x14x30.
    out_layer1 = tf.nn.max_pool(out_layer1, [1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME', name='s2_Covnet-out_layer1-pool')
    
    #Layer 2: Convolutional. Output = 10x10x64.
    wc2 = tf.Variable(tf.truncated_normal([5, 5, 30, 64], dtype=tf.float32, mean=mu, stddev=sigma), name='s2_Covnet-wc2')
    bc2 = tf.Variable(tf.zeros([64], dtype=tf.float32), name='s2_Covnet-bc2')
    strides = [1, 1, 1, 1]
    padding = 'VALID'
    out_layer2 = tf.nn.bias_add(tf.nn.conv2d(out_layer1, wc2, strides, padding), bc2, name='s2_Covnet-out_layer2')
    
    #Activation.
    out_layer2 = tf.nn.relu(out_layer2, name='s2_Covnet-out_layer2-dropout')
    
    #Pooling. Input = 10x10x64. Output = 5x5x64.
    out_layer2 = tf.nn.max_pool(out_layer2, [1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME', name='s2_Covnet-out_layer2-pool')

    #Flatten. Input = (the output of both convolutional layers). 
    out_conv_layers = tf.concat(1, [flatten(out_layer1), flatten(out_layer2)])
    
    #Layer 3: Fully Connected. Input = 7480. Output = 120.
    wf1 = tf.Variable(tf.truncated_normal([7480, 120], dtype=tf.float32, mean=mu, stddev=sigma), name='s2_Covnet-wf1')
    bf1 = tf.Variable(tf.zeros([120], dtype=tf.float32), name='s2_Covnet-bf1')
    out_layer3 = tf.add(tf.matmul(out_conv_layers, wf1), bf1, name='s2_Covnet-out_layer3')
    
    #Activation.
    out_layer3 = tf.nn.relu(out_layer3, name='s2_Covnet-out_layer3-relu')
    
    #Dropout
    out_layer3 = tf.nn.dropout(out_layer3, keep_prob, name='s2_Covnet-out_layer3-dropout')
        

    #Classifier, Input:120, output: n_classes
    wf2 = tf.Variable(tf.truncated_normal([120, n_classes], dtype=tf.float32, mean=mu, stddev=sigma), name='s2_Covnet-wf2')
    bf2 = tf.Variable(tf.zeros([n_classes], dtype=tf.float32), name='s2_Covnet-bf2')
    classifier = tf.add(tf.matmul(out_layer3, wf2), bf2)
    logits = classifier
    
    
    return logits

In [ ]:
#Features and labels
x = tf.placeholder(tf.float32, (None, image_shape[0], image_shape[1], image_shape[2]))
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, n_classes)

#keeping probability for the dropout function
keep_prob = tf.placeholder(tf.float32)

In [ ]:
#Training pipeline
rate = 0.0005

#Select which architecture to use for training and evaluation
#logits = LeNet(x, keep_prob)
logits = s2_Covnet(x, keep_prob)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

In [ ]:
#Evaluating accuracy of the model
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: 1})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [ ]:
### Training and validating the model
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, X_train_n, y_train = shuffle(X_train, X_train_n, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train_n[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: 0.5})
        
        training_accuracy = evaluate(X_train_n, y_train)
        validation_accuracy = evaluate(X_valid_n, y_valid)
        print("EPOCH {} ...".format(i+1))
        print("Training Accuracy = {:.3f}".format(training_accuracy))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
        
    saver.save(sess, './Traffic_Sign_Classifier_LeNet')
    print("Model saved")

In [ ]:
#Testing the model
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    #tf.train.Saver.restore(sess, './Traffic_Sign_Classifier_LeNet')
    #new_saver = tf.train.import_meta_graph('./Traffic_Sign_Classifier_LeNet.meta')
    #new_saver.restore(sess, tf.train.latest_checkpoint('./'))
    
    test_accuracy = evaluate(X_test_n, y_test)
    print("Test Accuracy = {:.3f}".format(test_accuracy))

In [ ]:
#Read the csv file with the meaning of each label
import csv

#obtaining messages meaning from csv file
signnames = csv.DictReader(open('signnames.csv'))
signnameslist = []                            
for row in signnames:
    signnameslist.append(row['SignName'])
    


In [ ]:
#evaluate and plot random plots of the training set
import random
import numpy as np
import matplotlib.pyplot as plt
import math 

n_visualize = 6 #number of images to be plotted
n_columns = 2 #number of columns in the figure
n_rows = math.ceil(n_visualize/n_columns) #number of rows in the figure

plt.figure(figsize=(15,15))
plt.suptitle('Testing set sample', fontsize=16)

for i in range(n_visualize):
    #obtain random images from test set
    index = random.randint(0,len(X_test))
    test_img, test_img_n, test_label = X_test[index], X_test_n[index], y_test[index]
 
    #evaluate selected image  
    with tf.Session() as sess:
        saver.restore(sess, tf.train.latest_checkpoint('.')) 
        output = sess.run(logits, feed_dict={x: [test_img_n], keep_prob: 1})
    #Analyzing result
    output = np.argmax(output)
    output_msg = signnameslist[output]
    actual_msg = signnameslist[test_label]
    
    plt.subplot(n_rows, n_columns, i+1)
    plt.imshow(X_test[index])
    plt.title((('Proposed classification: {}:'.format(output_msg) + '\n' + 'Actual classification: {}'.format(actual_msg))))

plt.subplots_adjust(top=0.9)
plt.savefig('Images/testing_set_sample.jpg')
plt.show()

---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
### Load the images and plot them here.
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2

#read files
files = os.listdir('Additional_test_images/')
for file in files:
    if(file[-4:].upper() != '.JPG'):
        files.remove(file)

n_imgs = len(files) #number of images in file
n_columns = 2 #number of columns in the figure
n_rows = math.ceil(n_imgs/n_columns) #number of rows in the figure
imgs = []

plt.figure(figsize=(6*n_columns,4*n_rows))
plt.suptitle('Additional Images', fontsize=16)

#read and show additional images for testing
for i in range(n_imgs):
    
    #reading images
    img = mpimg.imread('Additional_test_images/' + files[i])
    
    #resizing images to the shape of the training set
    img = cv2.resize(img, (image_shape[0],image_shape[1]))
    
    imgs.append(img)
    
    #outputing images
    plt.subplot(n_rows, n_columns, i+1)
    plt.imshow(img)
    plt.title('Sign: ' + files[i][:-4]) #the name of each image is it's meaning (removing .jpg)

plt.subplots_adjust(top=0.95)
plt.savefig('Images/additional_images.jpg')
plt.show()

In [ ]:
#Read the csv file with the meaning of each label
import csv

#obtaining message meaning from csv file
signnames = csv.DictReader(open('signnames.csv'))
signnameslist = []                            
for row in signnames:
    signnameslist.append(row['SignName'])
    

### Predict the Sign Type for Each Image

In [ ]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Preprocessing data

#normalizing the image 
if(convToGray & (imgs[0].shape[2] == 3)): 
    #apply_grayscaling prior to normalizing
    imgs_n = normalize(rgb2gray(imgs))
else:
    #only normalizing the images
    imgs_n = normalize(imgs)
    

In [ ]:
#evaluate output

#run evaluating model
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.')) 
    output = sess.run(logits, feed_dict={x: imgs_n, keep_prob: 1})

#converting the logits obtained to message list
output_msg = []
for i in range(output.shape[0]):
    output_msg.append(signnameslist[np.argmax(output[i])])

In [ ]:
#visualizing results
plt.figure(figsize=(6*n_columns,4*n_rows))
plt.suptitle('Additional Images result', fontsize=16)

for i in range(n_imgs):
    #obtain random images from test set
    plt.subplot(n_rows, n_columns, i+1)
    plt.imshow(imgs[i])
    plt.title("Result: {}".format(output_msg[i]))

plt.subplots_adjust(top=0.95)
plt.savefig('Images/additional_images_result.jpg')
plt.show()

### Analyze Performance

In [ ]:
### Calculate the accuracy for the new images. 
import re

#Calculating number of correct outputs
correct = 0
for i in range(n_imgs):
    prediction = re.sub('/', '', output_msg[i])
    actual = files[i][:-4] 
    if (prediction.upper() == actual.upper()):
        correct = correct + 1

accuracy = (correct / n_imgs) * 100
print('The accuracy of the prediction of the additional images is: {}%'.format(accuracy))

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tf.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 

#obtaining the softmax probabolties
with tf.Session() as sess:
    top_output = sess.run(tf.nn.top_k(tf.nn.softmax(output), k=5))

plt.figure(figsize=(20,4*n_imgs))
plt.suptitle('Softmax Probabilities for additional images', fontsize=16)

#displaying the softmax probabilties for all images
for i in range(0,len(top_output[0])):
    #print('The top 5 softmax probabilities for image: {}  are: {}'.format(i+1, top_output[0][i]))
    output_msgs =[]
    for output_msg in top_output[1][i]:
        output_msgs.append(signnameslist[output_msg])
    #print('The corresponding messages for image {}: are: {}'.format(i+1, output_msgs))
    plt.subplot(n_imgs, 2, (2*i)+1)
    plt.imshow(imgs[i])
    plt.subplot(n_imgs, 2, (2*i)+2)
    #print(len(top_output[0][i]))
    plt.barh(range(5), top_output[0][i])
    plt.yticks(range(5), output_msgs)
    plt.gca().invert_yaxis()

plt.subplots_adjust(top=0.95)
plt.savefig('Images/softmax_probabilities.jpg')
plt.show()

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

---

## Step 4 (Optional): Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it may be having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess, feed_dict={x: image_input, keep_prob: 1})
    print(activation.shape)
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(40,40))
    for featuremap in range(featuremaps):
        plt.subplot(10,7, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")

In [ ]:
#selecting and showing an image
import random

index = random.randint(0,n_train)
trial_image = X_train[index]
trial_image_n = X_train_n[index]

plt.imshow(trial_image)
plt.savefig('Images/vis_sample_img.jpg')
plt.show()

In [ ]:
#Visualizing first convolutional layer

with tf.Session() as sess:
    # Convolution (layer 1 after 'tf.nn.conv2d' operation)
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    conv1 = sess.graph.get_tensor_by_name('s2_Covnet-out_layer1:0')
    outputFeatureMap([trial_image_n], conv1)
    plt.suptitle('Visualization of first covnet layer', fontsize=22)
    plt.subplots_adjust(top=0.95)
    plt.savefig('Images/first_covnet_layer')
    plt.show()

In [ ]:
#Visualizing second convolutional layer
with tf.Session() as sess:
    # Convolution (layer 1 after 'tf.nn.conv2d' operation)
    saver.restore(sess, tf.train.latest_checkpoint('.'))
    conv1 = sess.graph.get_tensor_by_name('s2_Covnet-out_layer2:0')
    outputFeatureMap([trial_image_n], conv1)
    plt.suptitle('Visualization of second covnet layer', fontsize=22)
    plt.subplots_adjust(top=0.95)
    plt.savefig('Images/second_covnet_layer')
    plt.show()